In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from googleapiclient.discovery import build
import isodate
import creds
from sqlalchemy import create_engine
import db_creds

In [2]:
api_key  = creds.api_key
#channel_id = 'UCBJycsmduvYEL83R_U4JriQ'

# get multiple channels details

channel_ids = [     
               'UCBJycsmduvYEL83R_U4JriQ', # Mkbhd
               'UCMiJRAwDNSNzuYeN2uWa0pA', # Mrwhosetheboss
               'UCSOpcUkE-is7u7c4AkLgqTw', # Mrmobile
               'UCsTcErHg8oDvUnTzoqsYeNw', # Unbox Therapy
               'UCO2WJZKQoDW4Te6NHx4KfTg', # Geekyranjit
               'UCEPL07qzVsOcHd3sMUws65g', # Trakin Tech
               'UCXUJJNoP1QupwsYIWFXmsZg', # Tech Burner
               'UCS261lQDXjMvc8Jck-2CQEg', # Gyan therapy
               'UCvpfclapgcuJo0M_x65pfRw', # Beebom
               'UCdoDYkupTElV62K7ttA_E4w', # Tech Bar
               'UClVIlK8QHZ2PFkXF97bA0lg', # C4etech
               'UCn7DATpF0Nk-YJmjw4gMwfg', # Trakin Tech English
               'UCXuqSBlHAE6Xw-yeJA0Tunw' # Linus Tech Tips
               ]

youtube = build('youtube','v3',developerKey=api_key)

### Function to get Channel's stats

In [3]:
def get_channel_stats(youtube,channel_ids):
   
    all_data = []

    request = youtube.channels().list(part='snippet,contentDetails,statistics',id=','.join(channel_ids))

    response = request.execute()
    
    for i in range(len(response['items'])):

        data = dict(channel_name = response['items'][i]['snippet']['title'],
                    subscribers = response['items'][i]['statistics']['subscriberCount'],
                    views = response['items'][i]['statistics']['viewCount'],
                    total_videos = response['items'][i]['statistics']['videoCount'],
                    playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads']
                )
        all_data.append(data)
    
    return all_data

In [4]:
op_data = get_channel_stats(youtube,channel_ids)

In [5]:
df = pd.DataFrame(op_data)
df

,channel_name,subscribers,views,total_videos,playlist_id
0,Geekyranjit,3310000,823902382,3294,UUO2WJZKQoDW4Te6NHx4KfTg
1,TechBar,4280000,949701479,1201,UUdoDYkupTElV62K7ttA_E4w
2,Tech Burner,11000000,1824105226,1171,UUXUJJNoP1QupwsYIWFXmsZg
3,Linus Tech Tips,15400000,7320032614,6599,UUXuqSBlHAE6Xw-yeJA0Tunw
4,Marques Brownlee,17600000,3775158921,1588,UUBJycsmduvYEL83R_U4JriQ
5,Trakin Tech,13700000,2385135803,3712,UUEPL07qzVsOcHd3sMUws65g
6,Mrwhosetheboss,16600000,4343187185,1615,UUMiJRAwDNSNzuYeN2uWa0pA
7,Beebom,2990000,743409385,1116,UUvpfclapgcuJo0M_x65pfRw
8,C4ETech English,1800000,436522057,3246,UUlVIlK8QHZ2PFkXF97bA0lg
9,Gyan Therapy,2800000,594148991,1109,UUS261lQDXjMvc8Jck-2CQEg


### Function to get video IDs

In [6]:
#playlist_id = df.loc[df['channel_name']=='Beebom','playlist_id'].iloc[0]

playlist_ids = []

for i in df['playlist_id'].values:
    playlist_ids.append(i)

playlist_ids   

['UUO2WJZKQoDW4Te6NHx4KfTg',
 'UUdoDYkupTElV62K7ttA_E4w',
 'UUXUJJNoP1QupwsYIWFXmsZg',
 'UUXuqSBlHAE6Xw-yeJA0Tunw',
 'UUBJycsmduvYEL83R_U4JriQ',
 'UUEPL07qzVsOcHd3sMUws65g',
 'UUMiJRAwDNSNzuYeN2uWa0pA',
 'UUvpfclapgcuJo0M_x65pfRw',
 'UUlVIlK8QHZ2PFkXF97bA0lg',
 'UUS261lQDXjMvc8Jck-2CQEg',
 'UUSOpcUkE-is7u7c4AkLgqTw',
 'UUn7DATpF0Nk-YJmjw4gMwfg',
 'UUsTcErHg8oDvUnTzoqsYeNw']

In [7]:
def get_video_ids(youtube,playlist_ids):

    video_ids = []

    for playlist_id in playlist_ids:

        request = youtube.playlistItems().list(part='contentDetails',playlistId=playlist_id,maxResults=50)

        response = request.execute()

        for i in range(len(response['items'])):
            video_ids.append(response['items'][i]['contentDetails']['videoId'])

        

        next_page_token = response.get('nextPageToken')
        more_pages = True

        while more_pages:
            if next_page_token is None:
                more_pages = False
            else:
                request = youtube.playlistItems().list(part='contentDetails',
                                                        playlistId=playlist_id,
                                                        maxResults=50,
                                                        pageToken=next_page_token)
                response = request.execute()

                for i in range(len(response['items'])):
                    video_ids.append(response['items'][i]['contentDetails']['videoId'])
                
            next_page_token = response.get('nextPageToken')
    return video_ids

In [8]:
video_ids= get_video_ids(youtube,playlist_ids)
len(video_ids)

27673

### Function to get video details 

In [9]:
def get_video_details(youtube,video_ids):
    
    all_video_stats = [] 

    for i in range(0, len(video_ids), 50):
        requests = youtube.videos().list(part='snippet,statistics,contentDetails,topicDetails',id=','.join(video_ids[i:i+50]))
        response = requests.execute()
        
        for video in response['items']:
            video_stats = dict(channel_name = video['snippet'].get('channelTitle'),
                               title  = video['snippet'].get('title'),
                               published_date = video['snippet'].get('publishedAt'),
                               tags = video['snippet'].get('tags'),
                               views = video['statistics'].get('viewCount',0),
                               likes = video['statistics'].get('likeCount',0),
                               comments = video['statistics'].get('commentCount',0),  
                               favorite_count = video['statistics'].get('favoriteCount',0),    
                               duration = video['contentDetails'].get('duration'),
                               definition = video['contentDetails'].get('definition'),
                               caption = video['contentDetails'].get('caption')
                              )
            all_video_stats.append(video_stats)

    return all_video_stats

In [10]:
vid = get_video_details(youtube,video_ids)

In [11]:
df_vid = pd.DataFrame(vid)
df_vid.head(3)

,channel_name,title,published_date,tags,views,likes,comments,favorite_count,duration,definition,caption
0,Geekyranjit,Moto Edge 40 Neo - The Game Changer | Premium...,2023-09-22T13:34:04Z,"[moto edge 40 neo, motorola edge 40 neo, edge ...",359905,10517,1100,0,PT15M11S,hd,false
1,Geekyranjit,LG QNED (2023) - Good 120Hz 4K Smart TV,2023-09-20T07:27:03Z,"[LG QNED, LG OLED, LG 2023, LG smart tv, LG be...",58712,1184,96,0,PT9M42S,hd,false
2,Geekyranjit,Moto G54 5G Overview // The New Mid Range Cham...,2023-09-06T07:07:21Z,"[moto g54, moto g 54, motorola g54, moto g54 i...",254419,5965,844,0,PT14M37S,hd,false


### Fix data type

In [12]:
df_vid.dtypes

channel_name      object
title             object
published_date    object
tags              object
views             object
likes             object
comments          object
favorite_count    object
duration          object
definition        object
caption           object
dtype: object

### Change data type of columns

In [13]:
df_vid['published_date'] = pd.to_datetime(df_vid['published_date'])
df_vid['views'] = df_vid['views'].astype(int)
df_vid['likes'] = df_vid['likes'].astype(int)
df_vid['comments'] = df_vid['comments'].astype(int)
df_vid['favorite_count'] = df_vid['favorite_count'].astype(int)

#### Convert Duration column from YT time format to Seconds

In [14]:
new_dur=[]
for i in df_vid['duration'].values:
    dur = isodate.parse_duration(i)
    new_dur.append(dur.total_seconds())

df_vid['time_seconds'] = new_dur

### Get year and month from published_dat

In [15]:
df_vid['day'] = df_vid['published_date'].dt.date
df_vid['year'] = df_vid['published_date'].dt.year
df_vid['month'] = df_vid['published_date'].dt.month
df_vid.head(3)

,channel_name,title,published_date,tags,views,likes,comments,favorite_count,duration,definition,caption,time_seconds,day,year,month
0,Geekyranjit,Moto Edge 40 Neo - The Game Changer | Premium...,2023-09-22 13:34:04+00:00,"[moto edge 40 neo, motorola edge 40 neo, edge ...",359905,10517,1100,0,PT15M11S,hd,false,911.0,2023-09-22,2023,9
1,Geekyranjit,LG QNED (2023) - Good 120Hz 4K Smart TV,2023-09-20 07:27:03+00:00,"[LG QNED, LG OLED, LG 2023, LG smart tv, LG be...",58712,1184,96,0,PT9M42S,hd,false,582.0,2023-09-20,2023,9
2,Geekyranjit,Moto G54 5G Overview // The New Mid Range Cham...,2023-09-06 07:07:21+00:00,"[moto g54, moto g 54, motorola g54, moto g54 i...",254419,5965,844,0,PT14M37S,hd,false,877.0,2023-09-06,2023,9


### Get tag count

In [16]:
df_vid['tag_count'] = df_vid['tags'].apply(lambda x : 0 if x is None else len(x))

df_vid.head(3)

,channel_name,title,published_date,tags,views,likes,comments,favorite_count,duration,definition,caption,time_seconds,day,year,month,tag_count
0,Geekyranjit,Moto Edge 40 Neo - The Game Changer | Premium...,2023-09-22 13:34:04+00:00,"[moto edge 40 neo, motorola edge 40 neo, edge ...",359905,10517,1100,0,PT15M11S,hd,false,911.0,2023-09-22,2023,9,10
1,Geekyranjit,LG QNED (2023) - Good 120Hz 4K Smart TV,2023-09-20 07:27:03+00:00,"[LG QNED, LG OLED, LG 2023, LG smart tv, LG be...",58712,1184,96,0,PT9M42S,hd,false,582.0,2023-09-20,2023,9,14
2,Geekyranjit,Moto G54 5G Overview // The New Mid Range Cham...,2023-09-06 07:07:21+00:00,"[moto g54, moto g 54, motorola g54, moto g54 i...",254419,5965,844,0,PT14M37S,hd,false,877.0,2023-09-06,2023,9,8


In [17]:
df_vid.isnull().sum()

channel_name        0
title               0
published_date      0
tags              598
views               0
likes               0
comments            0
favorite_count      0
duration            0
definition          0
caption             0
time_seconds        0
day                 0
year                0
month               0
tag_count           0
dtype: int64

In [30]:
x=df_vid[(df_vid['channel_name']=='Linus Tech Tips') & (df_vid['year']==2023)]
x.sort_values(by='day',ascending=False)

,channel_name,title,published_date,tags,views,likes,comments,favorite_count,duration,definition,caption,time_seconds,day,year,month,tag_count
5668,Linus Tech Tips,Dell DIDN’T Screw Me Over - $1500 PC Secret Sh...,2023-10-01 17:37:23+00:00,"[PC Shop, Computer Shop, Apple, Dell, Lenovo, ...",952462,55628,3620,0,PT37M37S,hd,false,2257.0,2023-10-01,2023,10,18
5669,Linus Tech Tips,WAN:GO Is Gone Forever - WAN Show September 29...,2023-09-30 05:45:42+00:00,"[pcmr, building, competition, gamers, how to, ...",417598,8803,874,0,PT3H54M45S,hd,false,14085.0,2023-09-30,2023,9,14
5670,Linus Tech Tips,I’m Staying on AMD! - Radeon Challenge Pt. 2,2023-09-28 17:00:48+00:00,"[AMD, Challenge, GPU, Seasonic, Gaming]",999015,49066,5821,0,PT17M32S,hd,false,1052.0,2023-09-28,2023,9,5
5671,Linus Tech Tips,My YouTuber merch is not the best.. :(,2023-09-26 21:06:45+00:00,"[wendell, merch, level1techs, kvm, linus revie...",1359880,62988,2715,0,PT24M51S,hd,false,1491.0,2023-09-26,2023,9,7
5672,Linus Tech Tips,We Bought a Giant Movie CD Player and it’s AMA...,2023-09-24 18:16:03+00:00,"[laserdisc, vhs, crt, trying old tech, dvd, vi...",1445185,54351,3931,0,PT25M17S,hd,true,1517.0,2023-09-24,2023,9,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5940,Linus Tech Tips,Nvidia is lying to you - RTX 4070 Ti Full Review,2023-01-04 14:00:00+00:00,"[nvidia, gpu, graphics card, video card, gefor...",3430468,99730,8771,0,PT22M51S,hd,true,1371.0,2023-01-04,2023,1,40
5941,Linus Tech Tips,These Gaming Mice are WEIRD…,2023-01-04 06:00:13+00:00,None,2857727,158808,1297,0,PT1M,hd,false,60.0,2023-01-04,2023,1,0
5942,Linus Tech Tips,We've made some AWFUL videos... - Reacting to ...,2023-01-03 21:48:28+00:00,"[reactrion, reacting, worst, videos, lett, ltt...",2257399,114247,4074,0,PT24M24S,hd,true,1464.0,2023-01-03,2023,1,14
5943,Linus Tech Tips,This GPU is WIRELESS?! - Retro GTX 460 WHDI,2023-01-02 20:16:01+00:00,"[gpu, graphics card, video card, wireless, whd...",3191617,103986,3919,0,PT22M3S,hd,true,1323.0,2023-01-02,2023,1,28


In [31]:
len(df_vid)

27673

### Connect to PostgreSQL database and insert the df_vid data into the table

In [34]:
username=db_creds.username
hostname=db_creds.hostname
port=db_creds.port
pwd=db_creds.password
db_name=db_creds.dbname


engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(username,pwd,hostname,port,db_name))

connection = engine.connect()

df_vid.to_sql('youtube',con=engine,if_exists='replace',index=False)


# engine.commit()  ## commit the changes to the database

df.to_sql('yt_channel_stats',con=engine,if_exists='replace',index=False)
# engine.commit()

# connection.commit()  
connection.close()


AttributeError: 'Connection' object has no attribute 'commit'